In [38]:
import sys
import os
import numpy as np
from pprint import pprint

In [39]:
sys.path.append('..')

from endure.lsm import (
    Cost,
    ClassicGen,
    LSMBounds,
    Workload,
    System,
    Policy,
    LSMDesign
)
from endure.solver import ClassicSolver

# Defining Our Environment

Let us first create our environment to tune an LSM tree. We start with the `LSMBounds` class, which contains the minimum and maximum values for each variable. We can start with the default initialization.

The bounds will be passed in other places to keep all values in check.

In [40]:
bounds = LSMBounds()
pprint(bounds)

LSMBounds(max_considered_levels=20,
          bits_per_elem_range=(1, 10),
          size_ratio_range=(2, 31),
          page_sizes=(4, 8, 16),
          entry_sizes=(1024, 2048, 4096, 8192),
          memory_budget_range=(5.0, 20.0),
          selectivity_range=(1e-07, 1e-09),
          elements_range=(100000000, 1000000000))


We will create a data generator to create some random workloads and random designs.

In [41]:
# generator is initialized with just bounds, you can add a random seed to make results reproducible
gen = ClassicGen(bounds, seed=42)

In [42]:
workload = gen.sample_workload()
pprint(workload)

Workload(z0=np.float64(0.439),
         z1=np.float64(0.335),
         q=np.float64(0.08499999999999996),
         w=np.float64(0.14100000000000001))


In [43]:
system = gen.sample_system()
pprint(system)

System(entry_size=np.int64(1024),
       selectivity=9.067644255912269e-08,
       entries_per_page=np.float64(128.0),
       num_entries=np.int64(762177133),
       mem_budget=19.63433527455134,
       phi=1.0)


In [44]:
design = gen.sample_design(system)
pprint(design)

LSMDesign(bits_per_elem=np.float64(15.569),
          size_ratio=np.int64(24),
          policy=<Policy.Tiering: 0>,
          kapacity=())


You can see designs have to obey restrictions on the system. For example, the `design.bits_per_elem < system.mem_budget` as we cannot allocate more memory than we have.

# Creating a tuning

Let us try to create a tuning now. We have our environment set, now we can create a solver class and ask it for a tuning.

In [45]:
solver = ClassicSolver(bounds)

In [46]:
design, scipy_opt_obj = solver.get_nominal_design(system, workload)
pprint(design)

LSMDesign(bits_per_elem=np.float64(9.043625464584622),
          size_ratio=np.float64(30.0),
          policy=<Policy.Leveling: 1>,
          kapacity=())


The solver will spit out both a design, and the raw scipy optimizer object. This is just useful as a sanity check to see if the optimization process terminated successfully or not.

In [47]:
scipy_opt_obj

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.5466902873359518
       x: [ 9.044e+00  3.000e+01]
     nit: 21
     jac: [-1.093e-05 -1.064e-04]
    nfev: 63
    njev: 21

Sometimes you may find that the robust solver will get stuck and have issues finding the correct tuning (generally you'll see this in the form of errors, not all of them indicate an unsuccesful tuning).

In [48]:
robust_design, scipy_opt_robust_obj = solver.get_robust_design(system, workload, rho=1)
pprint(robust_design)

LSMDesign(bits_per_elem=np.float64(2.805449723312165),
          size_ratio=np.float64(30.0),
          policy=<Policy.Leveling: 1>,
          kapacity=())


/Users/ndhuynh/repos/private_lsm_tuning/notebook/../endure/solver/util.py:19: RuntimeWarning: overflow encountered in exp
  return np.exp(input) - 1
/Users/ndhuynh/miniconda3/envs/axe/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


In [49]:
scipy_opt_robust_obj

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.3986303056113707
       x: [ 2.805e+00  3.000e+01  3.707e-01  1.028e+00]
     nit: 21
     jac: [-2.766e-05 -7.179e-03  2.897e-03  1.429e-03]
    nfev: 108
    njev: 21

What we've found is changing the initial starting point to the original nominal tuning can help alleviate these issues.

In [50]:
robust_design, _ = solver.get_robust_design(
    system=system,
    workload=workload,
    rho=1,
    init_args=[design.bits_per_elem, design.size_ratio, 1, 1]
)

In [51]:
robust_design

LSMDesign(bits_per_elem=np.float64(2.8045314892604845), size_ratio=np.float64(30.0), policy=<Policy.Leveling: 1>, kapacity=())

# Evaluating Tunings

To evaluate our designs, we can use the cost model to get immediate feedback. Treat this as if we were just running a simulation in place of executing a workload on a DB (which we'll do at some point!).

Here if we apply the initial workload on the nominal and robust tuning, we should expect the robust tuning to do worse, as the executed workload is the same as the tuned workload.

In [52]:
cost = Cost(bounds.max_considered_levels)

In [53]:
cost.calc_cost(robust_design, system, workload)

0.659779650632332

In [54]:
cost.calc_cost(design, system, workload)

0.5466902873359518

Lets try applying a different workload to both designs. We can check how far away workloads are using the KL-Div. We will use scipys standard implementation [rel_tr (docs)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.rel_entr.html#scipy.special.rel_entr).

In [55]:
from scipy.special import rel_entr

In [56]:
# Note that KL divergence is NOT symmetric, in other words div(a, b) != div(b, a)
def kl_div(w_hat: Workload, w_init: Workload):
    w_left = np.array([w_hat.z0, w_hat.z1, w_hat.q, w_hat.w])
    w_right = np.array([w_init.z0, w_init.z1, w_init.q, w_init.w])
    return np.sum(rel_entr(w_left, w_right))

In [57]:
new_workload = Workload(z0=0.1, z1=0.1, q=0.7, w=0.1)
pprint(new_workload)

Workload(z0=0.1, z1=0.1, q=0.7, w=0.1)


In [67]:
rho = kl_div(new_workload, workload)
print(f'{rho.item()=}')

rho.item()=1.1727124272557485


In [58]:
cost.calc_cost(robust_design, system, new_workload)

1.3864055077210482

In [59]:
cost.calc_cost(design, system, new_workload)

1.4540570024583315

We can see the new owrk